In [ ]:
# 1 load xml； 2 load jpg ；3 haar gray ；4 detect； 5 对于检测出的结果进行遍历，绘制，draw
import cv2
import numpy as np
# step1:load xml 1 file name 完成xml文件的引入，参数就是文件的名称
face_xml = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')#训练好的人脸识别的分类器
eye_xml = cv2.CascadeClassifier('haarcascade_eye.xml')#眼睛识别的训练好的adaboost分类器
# step2:load jpg加载图片
img = cv2.imread('face.jpg')
cv2.imshow('src',img)
# step3:灰度转化haar gray 需要把b,g,r图片转化为灰度图片
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# detect faces 1 data灰度图片的数据； 2 scale比例缩放； 3 目标大小，比如不能小于5个像素  检测出的人脸
faces = face_xml.detectMultiScale(gray,1.3,5)
print('face=',len(faces))#检测当前的人脸
# draw给每一个人脸画一个框
for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)#绘制的数据； 绘制的起始目标；宽度，高度；颜色；线条宽度
    roi_face = gray[y:y+h,x:x+w]#表示已经找到的人脸的范围
    roi_color = img[y:y+h,x:x+w]#再记录一个彩色的人脸
    # 1 gray
    eyes = eye_xml.detectMultiScale(roi_face)
    print('eye=',len(eyes))#眼睛个数
    #绘制一下眼睛，还是使用for循环，眼睛开始位置的x,y坐标，宽，高
    #for (e_x,e_y,e_w,e_h) in eyes:
        #cv2.rectangle(roi_color,(e_x,e_y),(e_x+e_w,e_y+e_h),(0,255,0),2)
cv2.imshow('dst',img)
cv2.waitKey(0)


face= 1
eye= 2


In [ ]:
#adaboost 训练
# 1 初始化数据权值分布
# 苹果 苹果 苹果 香蕉
# 0.1 0.1 0.1 0.1 
# 2 遍历阈值 p
# minP t 
# 3 G1（x）
# 4 权值分布 update
# 0.2 0.2 0.2 0.7
## 训练终止条件：1 for count 2 p

In [ ]:
#haar + adaboost face
# eg：苹果 苹果 苹果 香蕉想通过这个分类器把苹果区分出来
# 训练出结果：0.1  0.1 0.1 0.5，再进行一次，会使得负样本得到加强。
# 训练终止条件：1 for count ；2 检测概率，最小的检测概率，p
# 1 分类器的结构 2 adaboost 计算过程 3 opencv自带的，xml 文件结构
# haar> T1 and haar>T2  2个强分类器，一般adaboost个数是15-20个强分类器 （Haar特征需要进行阈值判决），每一级都通过才认可

# 1 分类器的结构
# 3个强分类器 1 x1 t1 2 x2 t2 3 x3 t3
# x1>t1 and x2>t2 and x3>t3  目标-》苹果
# 强分类器作用：判决
# 弱分类器结构
# 弱分类器作用：计算强分类器特征x1 x2 x3
# x2 = sum（y1,y2,y3）

# y1 弱分类器特征是如何计算的？归结于node结点，每一个弱分类器支持三个haar特征
# node
# 3个haar-》 node
# 1node haar1 > nodeT1（阈值判决）   z1 = a1
# 1node haar1 < nodeT1   z1 = a2
# Z（整的node结点） = sum(z1,z2,z3)>T（弱分类器判决门限） y1 = AA（三个之和）
# Z = sum(z1,z2,z3)<T y1 = BB

# haar->Node z1 z2 z3 Z=sum(z1,z2,z3)每一个node结点对应一个haar特征，opencv最多有三个haar特征,所以最多三个node
# Z>T y1 y2 y3(第二层：弱分类器)
# x = sum(y1,y2,y3) > T1 obj（强分类器）【目标：就是要判决出苹果】


In [ ]:
#haar 1 什么是haar？特征 = 像素 运算 -》结果 （具体值 向量 矩阵 多维）
# 2 如何利用特征 区分目标？ 阈值判决 
# 3 得到判决？机器学习 
# 1 特征 2 判决 3 得到判决

# 公式推导 1  -2
# 特征 = 整个区域*权重1 + 黑色*权重2 = （黑+白）*1+黑*（-2）=
# = 黑+白-2黑 = 白-黑

# 1 haar模版 上下 左右 image size 模版 size 100*100 10*10 100次 step = 10
# 1 100*100 2 10*10 3 step 10 4 模版1
# 模版 滑动 缩放 10*10 缩放之后，变成11*11 20级（总共可能需要缩放20次）

# 举例 图片大小：1080*720 ； 每次滑动两个像素，所以步长是2step2； 原始模板大小：10*10 
# 计算量 = 14模版*20缩放*（1080/2*720/2）*（100点+- ） = 50-100亿
# （50-100）*15（不能低于15） = 1000亿次才能把Haar特征计算完，计算量太大了

# A 1 B 1 2 C 1 3 D 1 2 3 4
# 4 = A-B-C+D = 1+1+2+3+4 - 1 -2 - 1 -3 = 4 (3+-)